In [ ]:
import unicodedata
import re
import pickle
import os
import gc
import copy
import time
import joblib
from pprint import pprint
import random
from collections import defaultdict, Counter
from tqdm import tqdm
import string
from typing import List, Dict, Tuple
from datetime import datetime, timedelta, timezone
import math

import numpy as np
import pandas as pd
import polars as pl
import cudf
from matplotlib import pyplot as plt

from gensim.models import Word2Vec

import warnings

warnings.filterwarnings("ignore")

os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

print(os.cpu_count())

In [ ]:
# グローバル変数設定

ROOT = ""  # コンペ用ディレクトリ
OUTPUT_DIR = ""
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [ ]:
# メモリ削減（数値カラムのみ）
def reduce_mem_usage_for_numeric(df):
    """iterate through  the numeric columns of a dataframe and modify the data type
    to reduce memory usage.
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print("Memory usage of dataframe is {:.2f} MB".format(start_mem))

    for col in df.columns:
        col_type = df[col].dtype

        if "int" in str(col_type) or "float" in str(col_type):
            c_min = df[col].min()
            c_max = df[col].max()
            if "int" in str(col_type):
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            elif "float" in str(col_type):
                # if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                #     df[col] = df[col].astype(np.float16)# サポート対象故
                if (
                    c_min > np.finfo(np.float32).min
                    and c_max < np.finfo(np.float32).max
                ):
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    print("Memory usage after optimization is: {:.2f} MB".format(end_mem))
    print("Decreased by {:.1f}%".format(100 * (start_mem - end_mem) / start_mem))

    return df    print("Decreased by {:.1f}%".format(100 * (start_mem - end_mem) / start_mem))

    return df    print("Decreased by {:.1f}%".format(100 * (start_mem - end_mem) / start_mem))

    return df    print("Decreased by {:.1f}%".format(100 * (start_mem - end_mem) / start_mem))

    return df

In [ ]:
# 理想時のcvを見ることができる。
def calc_candidates_recall(candidates, label, type):
    pred = candidates.groupby("session").aid.apply(set)
    pred = pred.reset_index()
    gt = label[label["type"] == type]

    gt_pred = gt.merge(pred, on="session", how="left")

    # negasamp後に、gtに紐づかないaidが出る可能性があるので、空のsetでfillna
    gt_pred["aid"] = gt_pred["aid"].apply(lambda d: d if isinstance(d, set) else set())

    gt_pred["hits"] = gt_pred.apply(
        lambda x: min(len(set(x["ground_truth"]) & x["aid"]), 20), axis=1
    )
    gt_pred["gt_count"] = gt_pred.apply(
        lambda x: min(len(x["ground_truth"]), 20), axis=1
    )
    print(gt_pred.hits.sum() / gt_pred.gt_count.sum())    gt_pred["gt_count"] = gt_pred.apply(
        lambda x: min(len(x["ground_truth"]), 20), axis=1
    )
    print(gt_pred.hits.sum() / gt_pred.gt_count.sum())    gt_pred["gt_count"] = gt_pred.apply(
        lambda x: min(len(x["ground_truth"]), 20), axis=1
    )
    print(gt_pred.hits.sum() / gt_pred.gt_count.sum())    gt_pred["gt_count"] = gt_pred.apply(
        lambda x: min(len(x["ground_truth"]), 20), axis=1
    )
    print(gt_pred.hits.sum() / gt_pred.gt_count.sum())

In [ ]:
valid_label = pd.read_pickle(f"{ROOT}/data/input/valid_label_1week.pkl")
valid_session = pd.read_pickle(f"{ROOT}/data/input/valid_trimed_session_1week.pkl")

In [ ]:
test = pd.read_pickle(f"{ROOT}/data/input/processed_data2/test.pkl")

# make candidate train

## click

In [ ]:
valid_candidate1 = pd.read_pickle(
    f"{ROOT}/data/output/candidates/GRU4Rec/train/exp/001/gru4rec_candidates.pkl"
)
valid_candidate2 = pd.read_pickle(
    f"{ROOT}/data/output/candidates/past_events/train/exp/005/past_action_candidates_max100.pkl"
)
valid_candidate3 = pd.read_pickle(
    f"{ROOT}/data/output/candidates/covisitation_matrix/train/click_only_matrix_candidates_max40.pkl"
)
valid_candidate4 = pd.read_pickle(
    f"{ROOT}/data/output/candidates/covisitation_matrix/train/exp/026/time_decay_matrix_exp026_candidates_last10_suggest20.pkl"
)

valid_candidate5 = pd.read_pickle(
    f"{ROOT}/data/output/candidates/BPR/train/exp/005/BPR_candidates.pkl"
)

valid_candidate6 = pd.read_pickle(
    f"{ROOT}/data/output/candidates/SRGNN/train/exp/001/SRGNN_candidates.pkl"
)

valid_candidate7 = pd.read_pickle(
    f"{ROOT}/data/output/candidates/covisitation_matrix/train/exp/011/time_decay_matrix3_weight_click0.5_cart9_order0.5_candidates_last10_suggest20.pkl"
)

valid_candidate8 = pd.read_pickle(
    f"{ROOT}/data/output/candidates/itemCF/train/exp/003/item_cf_candidates_max40.pkl"
)

valid_candidate9 = pd.read_pickle(
    f"{ROOT}/data/output/candidates/covisitation_matrix/train/exp/009/time_decay_matrix_candidates_max40.pkl"
)

valid_candidate10 = pd.read_pickle(
    f"{ROOT}/data/output/candidates/IMF/train/exp/001/imf_candidates.pkl"
)

valid_candidate11 = pd.read_pickle(
    f"{ROOT}/data/output/candidates/w2v/train/exp/008/w2v_candidates.pkl"
)

valid_candidate12 = pd.read_pickle(
    f"{ROOT}/data/output/candidates/user_cf/train/exp/002/user_cf_candidates.pkl"
)
valid_candidate8 = pd.read_pickle(
    f"{ROOT}/data/output/candidates/itemCF/train/exp/003/item_cf_candidates_max40.pkl"
)

valid_candidate9 = pd.read_pickle(
    f"{ROOT}/data/output/candidates/covisitation_matrix/train/exp/009/time_decay_matrix_candidates_max40.pkl"
)

valid_candidate10 = pd.read_pickle(
    f"{ROOT}/data/output/candidates/IMF/train/exp/001/imf_candidates.pkl"
)

valid_candidate11 = pd.read_pickle(
    f"{ROOT}/data/output/candidates/w2v/train/exp/008/w2v_candidates.pkl"
)

valid_candidate12 = pd.read_pickle(
    f"{ROOT}/data/output/candidates/user_cf/train/exp/002/user_cf_candidates.pkl"
)
valid_candidate8 = pd.read_pickle(
    f"{ROOT}/data/output/candidates/itemCF/train/exp/003/item_cf_candidates_max40.pkl"
)

valid_candidate9 = pd.read_pickle(
    f"{ROOT}/data/output/candidates/covisitation_matrix/train/exp/009/time_decay_matrix_candidates_max40.pkl"
)

valid_candidate10 = pd.read_pickle(
    f"{ROOT}/data/output/candidates/IMF/train/exp/001/imf_candidates.pkl"
)

valid_candidate11 = pd.read_pickle(
    f"{ROOT}/data/output/candidates/w2v/train/exp/008/w2v_candidates.pkl"
)

valid_candidate12 = pd.read_pickle(
    f"{ROOT}/data/output/candidates/user_cf/train/exp/002/user_cf_candidates.pkl"
)
valid_candidate8 = pd.read_pickle(
    f"{ROOT}/data/output/candidates/itemCF/train/exp/003/item_cf_candidates_max40.pkl"
)

valid_candidate9 = pd.read_pickle(
    f"{ROOT}/data/output/candidates/covisitation_matrix/train/exp/009/time_decay_matrix_candidates_max40.pkl"
)

valid_candidate10 = pd.read_pickle(
    f"{ROOT}/data/output/candidates/IMF/train/exp/001/imf_candidates.pkl"
)

valid_candidate11 = pd.read_pickle(
    f"{ROOT}/data/output/candidates/w2v/train/exp/008/w2v_candidates.pkl"
)

valid_candidate12 = pd.read_pickle(
    f"{ROOT}/data/output/candidates/user_cf/train/exp/002/user_cf_candidates.pkl"
)

In [ ]:
# candidate_rank
valid_candidate1["gru4rec_score_ranking"] = valid_candidate1.groupby(
    "session"
).gru4rec_score.rank()
valid_candidate1["gru4rec_score_ranking"] = valid_candidate1[
    "gru4rec_score_ranking"
].astype(np.float32)

valid_candidate3["click_only_matrix_rank_ranking"] = valid_candidate3.groupby(
    "session"
).click_only_matrix_rank.rank()
valid_candidate3["click_only_matrix_rank_ranking"] = valid_candidate3[
    "click_only_matrix_rank_ranking"
].astype(np.float32)

valid_candidate5["BPR_score_ranking"] = valid_candidate5.groupby(
    "session"
).BPR_score.rank()
valid_candidate5["BPR_score_ranking"] = valid_candidate5["BPR_score_ranking"].astype(
    np.float32
)

valid_candidate6["SRGNN_score_ranking"] = valid_candidate6.groupby(
    "session"
).SRGNN_score.rank()
valid_candidate6["SRGNN_score_ranking"] = valid_candidate6[
    "SRGNN_score_ranking"
].astype(np.float32)

valid_candidate8["item_cf_count_ranking"] = valid_candidate8.groupby(
    "session"
).item_cf_count.rank()
valid_candidate8["item_cf_count_ranking"] = valid_candidate8[
    "item_cf_count_ranking"
].astype(np.float32)

valid_candidate9["time_decay_matrix_count_ranking"] = valid_candidate9.groupby(
    "session"
).time_decay_matrix_count.rank()
valid_candidate9["time_decay_matrix_count_ranking"] = valid_candidate9[
    "time_decay_matrix_count_ranking"
].astype(np.float32)

valid_candidate10["imf_scores_ranking"] = valid_candidate10.groupby(
    "session"
).imf_scores.rank()
valid_candidate10["imf_scores_ranking"] = valid_candidate10[
    "imf_scores_ranking"
].astype(np.float32)

valid_candidate11["w2v_angular_distance_ranking"] = valid_candidate11.groupby(
    "session"
).w2v_angular_distance.rank()
valid_candidate11["w2v_angular_distance_ranking"] = valid_candidate11[
    "w2v_angular_distance_ranking"
].astype(np.float32)

valid_candidate12["user_cf_score_ranking"] = valid_candidate12.groupby(
    "session"
).user_cf_score.rank()
valid_candidate12["user_cf_score_ranking"] = valid_candidate12[
    "user_cf_score_ranking"
].astype(np.float32)].astype(np.float32)

valid_candidate8["item_cf_count_ranking"] = valid_candidate8.groupby(
    "session"
).item_cf_count.rank()
valid_candidate8["item_cf_count_ranking"] = valid_candidate8[
    "item_cf_count_ranking"
].astype(np.float32)

valid_candidate9["time_decay_matrix_count_ranking"] = valid_candidate9.groupby(
    "session"
).time_decay_matrix_count.rank()
valid_candidate9["time_decay_matrix_count_ranking"] = valid_candidate9[
    "time_decay_matrix_count_ranking"
].astype(np.float32)

valid_candidate10["imf_scores_ranking"] = valid_candidate10.groupby(
    "session"
).imf_scores.rank()
valid_candidate10["imf_scores_ranking"] = valid_candidate10[
    "imf_scores_ranking"
].astype(np.float32)

valid_candidate11["w2v_angular_distance_ranking"] = valid_candidate11.groupby(
    "session"
).w2v_angular_distance.rank()
valid_candidate11["w2v_angular_distance_ranking"] = valid_candidate11[
    "w2v_angular_distance_ranking"
].astype(np.float32)

valid_candidate12["user_cf_score_ranking"] = valid_candidate12.groupby(
    "session"
).user_cf_score.rank()
valid_candidate12["user_cf_score_ranking"] = valid_candidate12[
    "user_cf_score_ranking"
].astype(np.float32)].astype(np.float32)

valid_candidate8["item_cf_count_ranking"] = valid_candidate8.groupby(
    "session"
).item_cf_count.rank()
valid_candidate8["item_cf_count_ranking"] = valid_candidate8[
    "item_cf_count_ranking"
].astype(np.float32)

valid_candidate9["time_decay_matrix_count_ranking"] = valid_candidate9.groupby(
    "session"
).time_decay_matrix_count.rank()
valid_candidate9["time_decay_matrix_count_ranking"] = valid_candidate9[
    "time_decay_matrix_count_ranking"
].astype(np.float32)

valid_candidate10["imf_scores_ranking"] = valid_candidate10.groupby(
    "session"
).imf_scores.rank()
valid_candidate10["imf_scores_ranking"] = valid_candidate10[
    "imf_scores_ranking"
].astype(np.float32)

valid_candidate11["w2v_angular_distance_ranking"] = valid_candidate11.groupby(
    "session"
).w2v_angular_distance.rank()
valid_candidate11["w2v_angular_distance_ranking"] = valid_candidate11[
    "w2v_angular_distance_ranking"
].astype(np.float32)

valid_candidate12["user_cf_score_ranking"] = valid_candidate12.groupby(
    "session"
).user_cf_score.rank()
valid_candidate12["user_cf_score_ranking"] = valid_candidate12[
    "user_cf_score_ranking"
].astype(np.float32)].astype(np.float32)

valid_candidate8["item_cf_count_ranking"] = valid_candidate8.groupby(
    "session"
).item_cf_count.rank()
valid_candidate8["item_cf_count_ranking"] = valid_candidate8[
    "item_cf_count_ranking"
].astype(np.float32)

valid_candidate9["time_decay_matrix_count_ranking"] = valid_candidate9.groupby(
    "session"
).time_decay_matrix_count.rank()
valid_candidate9["time_decay_matrix_count_ranking"] = valid_candidate9[
    "time_decay_matrix_count_ranking"
].astype(np.float32)

valid_candidate10["imf_scores_ranking"] = valid_candidate10.groupby(
    "session"
).imf_scores.rank()
valid_candidate10["imf_scores_ranking"] = valid_candidate10[
    "imf_scores_ranking"
].astype(np.float32)

valid_candidate11["w2v_angular_distance_ranking"] = valid_candidate11.groupby(
    "session"
).w2v_angular_distance.rank()
valid_candidate11["w2v_angular_distance_ranking"] = valid_candidate11[
    "w2v_angular_distance_ranking"
].astype(np.float32)

valid_candidate12["user_cf_score_ranking"] = valid_candidate12.groupby(
    "session"
).user_cf_score.rank()
valid_candidate12["user_cf_score_ranking"] = valid_candidate12[
    "user_cf_score_ranking"
].astype(np.float32)

In [ ]:
valid_candidate1 = pl.from_pandas(valid_candidate1)
valid_candidate2 = pl.from_pandas(valid_candidate2)
valid_candidate3 = pl.from_pandas(valid_candidate3)
valid_candidate4 = pl.from_pandas(valid_candidate4)
valid_candidate5 = pl.from_pandas(valid_candidate5)
valid_candidate6 = pl.from_pandas(valid_candidate6)
valid_candidate7 = pl.from_pandas(valid_candidate7)
valid_candidate8 = pl.from_pandas(valid_candidate8)
valid_candidate9 = pl.from_pandas(valid_candidate9)
valid_candidate10 = pl.from_pandas(valid_candidate10)
valid_candidate11 = pl.from_pandas(valid_candidate11)
valid_candidate12 = pl.from_pandas(valid_candidate12)

In [ ]:
def cast_cols(df, columns):
    for col in columns:
        df = df.with_column(df.get_column(col).cast(pl.Int32))
    return df


valid_candidate1 = cast_cols(valid_candidate1, ["session", "aid"])
valid_candidate2 = cast_cols(valid_candidate2, ["session", "aid"])
valid_candidate3 = cast_cols(valid_candidate3, ["session", "aid"])
valid_candidate4 = cast_cols(valid_candidate4, ["session", "aid"])
valid_candidate5 = cast_cols(valid_candidate5, ["session", "aid"])
valid_candidate6 = cast_cols(valid_candidate6, ["session", "aid"])
valid_candidate7 = cast_cols(valid_candidate7, ["session", "aid"])
valid_candidate8 = cast_cols(valid_candidate8, ["session", "aid"])
valid_candidate9 = cast_cols(valid_candidate9, ["session", "aid"])
valid_candidate10 = cast_cols(valid_candidate10, ["session", "aid"])
valid_candidate11 = cast_cols(valid_candidate11, ["session", "aid"])
valid_candidate12 = cast_cols(valid_candidate12, ["session", "aid"])

In [ ]:
valid_candidates = valid_candidate1.join(
    valid_candidate2, on=["session", "aid"], how="outer"
)
del valid_candidate1, valid_candidate2
gc.collect()
print("end")

valid_candidates = valid_candidates.join(
    valid_candidate3, on=["session", "aid"], how="outer"
)
del valid_candidate3
gc.collect()
print("end")

valid_candidates = valid_candidates.join(
    valid_candidate4, on=["session", "aid"], how="outer"
)
del valid_candidate4
gc.collect()
print("end")

valid_candidates = valid_candidates.join(
    valid_candidate5, on=["session", "aid"], how="outer"
)
del valid_candidate5
gc.collect()
print("end")

valid_candidates = valid_candidates.join(
    valid_candidate6, on=["session", "aid"], how="outer"
)
del valid_candidate6
gc.collect()
print("end")

valid_candidates = valid_candidates.join(
    valid_candidate7, on=["session", "aid"], how="outer"
)
del valid_candidate7
gc.collect()
print("end")

valid_candidates = valid_candidates.join(
    valid_candidate8, on=["session", "aid"], how="outer"
)
del valid_candidate8
gc.collect()
print("end")

valid_candidates = valid_candidates.join(
    valid_candidate9, on=["session", "aid"], how="outer"
)
del valid_candidate9
gc.collect()
print("end")

valid_candidates = valid_candidates.join(
    valid_candidate10, on=["session", "aid"], how="outer"
)
del valid_candidate10
gc.collect()
print("end")

valid_candidates = valid_candidates.join(
    valid_candidate11, on=["session", "aid"], how="outer"
)
del valid_candidate11
gc.collect()
print("end")

valid_candidates = valid_candidates.join(
    valid_candidate12, on=["session", "aid"], how="outer"
)
del valid_candidate12
gc.collect()
print("end")print("end")

valid_candidates = valid_candidates.join(
    valid_candidate10, on=["session", "aid"], how="outer"
)
del valid_candidate10
gc.collect()
print("end")

valid_candidates = valid_candidates.join(
    valid_candidate11, on=["session", "aid"], how="outer"
)
del valid_candidate11
gc.collect()
print("end")

valid_candidates = valid_candidates.join(
    valid_candidate12, on=["session", "aid"], how="outer"
)
del valid_candidate12
gc.collect()
print("end")print("end")

valid_candidates = valid_candidates.join(
    valid_candidate10, on=["session", "aid"], how="outer"
)
del valid_candidate10
gc.collect()
print("end")

valid_candidates = valid_candidates.join(
    valid_candidate11, on=["session", "aid"], how="outer"
)
del valid_candidate11
gc.collect()
print("end")

valid_candidates = valid_candidates.join(
    valid_candidate12, on=["session", "aid"], how="outer"
)
del valid_candidate12
gc.collect()
print("end")print("end")

valid_candidates = valid_candidates.join(
    valid_candidate10, on=["session", "aid"], how="outer"
)
del valid_candidate10
gc.collect()
print("end")

valid_candidates = valid_candidates.join(
    valid_candidate11, on=["session", "aid"], how="outer"
)
del valid_candidate11
gc.collect()
print("end")

valid_candidates = valid_candidates.join(
    valid_candidate12, on=["session", "aid"], how="outer"
)
del valid_candidate12
gc.collect()
print("end")

In [ ]:
valid_candidates = valid_candidates.to_pandas()

In [ ]:
# 型を変える
valid_candidates.fillna(-999, inplace=True)  # 欠損うめ
valid_candidates = reduce_mem_usage_for_numeric(valid_candidates)

In [ ]:
# 最高時のcv見てみる。
for type in ["clicks"]:
    print(f"{type}_candidates_recall")
    calc_candidates_recall(valid_candidates, valid_label, type)

### simialrityスコア付与

In [ ]:
last_items = (
    valid_session.groupby("session")[["session", "aid"]].tail(1).reset_index(drop=True)
)
last_items = dict(zip(last_items["session"], last_items["aid"]))

last2_items = (
    valid_session.groupby("session")[["session", "aid"]].nth(-2).reset_index(drop=True)
)
last2_items = dict(zip(last2_items["session"], last2_items["aid"]))last2_items = (
    valid_session.groupby("session")[["session", "aid"]].nth(-2).reset_index(drop=True)
)
last2_items = dict(zip(last2_items["session"], last2_items["aid"]))last2_items = (
    valid_session.groupby("session")[["session", "aid"]].nth(-2).reset_index(drop=True)
)
last2_items = dict(zip(last2_items["session"], last2_items["aid"]))last2_items = (
    valid_session.groupby("session")[["session", "aid"]].nth(-2).reset_index(drop=True)
)
last2_items = dict(zip(last2_items["session"], last2_items["aid"]))

In [ ]:
# 共起
matrix_file = f"{ROOT}/data/output/candidates/covisitation_matrix/train/exp/026/time_decay_matrix2_weight_click1_cart1_order1_chunk100000.pkl"
with open(matrix_file, "rb") as f:
    next_AIDs = pickle.load(f)

matrix_scores = []
for session, candidate in tqdm(
    zip(valid_candidates["session"], valid_candidates["aid"]),
    total=len(valid_candidates),
):
    score = next_AIDs[last_items[session]][candidate]
    matrix_scores.append(score)
valid_candidates["time_decay_matrix_score"] = matrix_scores

matrix_scores = []
for session, candidate in tqdm(
    zip(valid_candidates["session"], valid_candidates["aid"]),
    total=len(valid_candidates),
):
    try:
        score = next_AIDs[last2_items[session]][candidate]
    except:
        score = -1
    matrix_scores.append(score)
valid_candidates["time_decay_matrix_score_last2"] = matrix_scores

del next_AIDs
gc.collect()        score = -1
    matrix_scores.append(score)
valid_candidates["time_decay_matrix_score_last2"] = matrix_scores

del next_AIDs
gc.collect()        score = -1
    matrix_scores.append(score)
valid_candidates["time_decay_matrix_score_last2"] = matrix_scores

del next_AIDs
gc.collect()        score = -1
    matrix_scores.append(score)
valid_candidates["time_decay_matrix_score_last2"] = matrix_scores

del next_AIDs
gc.collect()

In [ ]:
# itemcf
itemcf_file = f"{ROOT}/data/output/candidates/itemCF/train/exp/001/sim_item_corr.pkl"
with open(itemcf_file, "rb") as f:
    item_cf_sim = pickle.load(f)

cf_scores = []
for session, candidate in tqdm(
    zip(valid_candidates["session"], valid_candidates["aid"]),
    total=len(valid_candidates),
):
    score = item_cf_sim[last_items[session]][candidate]
    cf_scores.append(score)
valid_candidates["item_cf_score"] = cf_scores

# cf feature
cf_scores = []
for session, candidate in tqdm(
    zip(valid_candidates["session"], valid_candidates["aid"]),
    total=len(valid_candidates),
):
    try:
        score = item_cf_sim[last2_items[session]][candidate]
    except:
        score = -1
    cf_scores.append(score)
valid_candidates["item_cf_score_last2"] = cf_scores

del item_cf_sim
gc.collect()        score = -1
    cf_scores.append(score)
valid_candidates["item_cf_score_last2"] = cf_scores

del item_cf_sim
gc.collect()        score = -1
    cf_scores.append(score)
valid_candidates["item_cf_score_last2"] = cf_scores

del item_cf_sim
gc.collect()        score = -1
    cf_scores.append(score)
valid_candidates["item_cf_score_last2"] = cf_scores

del item_cf_sim
gc.collect()

In [ ]:
model = Word2Vec.load(f"{ROOT}/data/output/candidates/w2v/train/exp/008/word2vec.model")


def cos_sim(v1, v2):
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))


similaritys = []
for session, candidate in tqdm(
    zip(valid_candidates["session"], valid_candidates["aid"]),
    total=len(valid_candidates),
):
    similarity = cos_sim(model.wv[last_items[session]], model.wv[candidate])
    similaritys.append(similarity)

valid_candidates["w2v_similarity"] = similaritys    similaritys.append(similarity)

valid_candidates["w2v_similarity"] = similaritys    similaritys.append(similarity)

valid_candidates["w2v_similarity"] = similaritys    similaritys.append(similarity)

valid_candidates["w2v_similarity"] = similaritys

### 保存

In [ ]:
# castする
valid_candidates = reduce_mem_usage_for_numeric(valid_candidates)
valid_candidates.to_pickle(f"{OUTPUT_DIR}/valid_click_candidates.pkl")

## cart&order

In [ ]:
valid_candidate1 = pd.read_pickle(
    f"{ROOT}/data/output/candidates/GRU4Rec/train/exp/001/gru4rec_candidates.pkl"
)
valid_candidate2 = pd.read_pickle(
    f"{ROOT}/data/output/candidates/past_events/train/exp/005/past_action_candidates_max100.pkl"
)
valid_candidate3 = pd.read_pickle(
    f"{ROOT}/data/output/candidates/covisitation_matrix/train/balance_matrix_candidates_max40.pkl"
)
valid_candidate4 = pd.read_pickle(
    f"{ROOT}/data/output/candidates/covisitation_matrix/train/exp/026/time_decay_matrix_exp026_candidates_last10_suggest20.pkl"
)

valid_candidate5 = pd.read_pickle(
    f"{ROOT}/data/output/candidates/BPR/train/exp/005/BPR_candidates.pkl"
)

valid_candidate6 = pd.read_pickle(
    f"{ROOT}/data/output/candidates/SRGNN/train/exp/001/SRGNN_candidates.pkl"
)

valid_candidate7 = pd.read_pickle(
    f"{ROOT}/data/output/candidates/covisitation_matrix/train/exp/011/time_decay_matrix3_weight_click0.5_cart9_order0.5_candidates_last10_suggest20.pkl"
)

valid_candidate8 = pd.read_pickle(
    f"{ROOT}/data/output/candidates/itemCF/train/exp/003/item_cf_candidates_max40.pkl"
)

valid_candidate9 = pd.read_pickle(
    f"{ROOT}/data/output/candidates/covisitation_matrix/train/exp/009/time_decay_matrix_candidates_max40.pkl"
)

valid_candidate10 = pd.read_pickle(
    f"{ROOT}/data/output/candidates/IMF/train/exp/001/imf_candidates.pkl"
)

valid_candidate11 = pd.read_pickle(
    f"{ROOT}/data/output/candidates/w2v/train/exp/008/w2v_candidates.pkl"
)

valid_candidate12 = pd.read_pickle(
    f"{ROOT}/data/output/candidates/user_cf/train/exp/002/user_cf_candidates.pkl"
)
valid_candidate8 = pd.read_pickle(
    f"{ROOT}/data/output/candidates/itemCF/train/exp/003/item_cf_candidates_max40.pkl"
)

valid_candidate9 = pd.read_pickle(
    f"{ROOT}/data/output/candidates/covisitation_matrix/train/exp/009/time_decay_matrix_candidates_max40.pkl"
)

valid_candidate10 = pd.read_pickle(
    f"{ROOT}/data/output/candidates/IMF/train/exp/001/imf_candidates.pkl"
)

valid_candidate11 = pd.read_pickle(
    f"{ROOT}/data/output/candidates/w2v/train/exp/008/w2v_candidates.pkl"
)

valid_candidate12 = pd.read_pickle(
    f"{ROOT}/data/output/candidates/user_cf/train/exp/002/user_cf_candidates.pkl"
)
valid_candidate8 = pd.read_pickle(
    f"{ROOT}/data/output/candidates/itemCF/train/exp/003/item_cf_candidates_max40.pkl"
)

valid_candidate9 = pd.read_pickle(
    f"{ROOT}/data/output/candidates/covisitation_matrix/train/exp/009/time_decay_matrix_candidates_max40.pkl"
)

valid_candidate10 = pd.read_pickle(
    f"{ROOT}/data/output/candidates/IMF/train/exp/001/imf_candidates.pkl"
)

valid_candidate11 = pd.read_pickle(
    f"{ROOT}/data/output/candidates/w2v/train/exp/008/w2v_candidates.pkl"
)

valid_candidate12 = pd.read_pickle(
    f"{ROOT}/data/output/candidates/user_cf/train/exp/002/user_cf_candidates.pkl"
)
valid_candidate8 = pd.read_pickle(
    f"{ROOT}/data/output/candidates/itemCF/train/exp/003/item_cf_candidates_max40.pkl"
)

valid_candidate9 = pd.read_pickle(
    f"{ROOT}/data/output/candidates/covisitation_matrix/train/exp/009/time_decay_matrix_candidates_max40.pkl"
)

valid_candidate10 = pd.read_pickle(
    f"{ROOT}/data/output/candidates/IMF/train/exp/001/imf_candidates.pkl"
)

valid_candidate11 = pd.read_pickle(
    f"{ROOT}/data/output/candidates/w2v/train/exp/008/w2v_candidates.pkl"
)

valid_candidate12 = pd.read_pickle(
    f"{ROOT}/data/output/candidates/user_cf/train/exp/002/user_cf_candidates.pkl"
)

In [ ]:
# candidate_rank
valid_candidate1["gru4rec_score_ranking"] = valid_candidate1.groupby(
    "session"
).gru4rec_score.rank()
valid_candidate1["gru4rec_score_ranking"] = valid_candidate1[
    "gru4rec_score_ranking"
].astype(np.float32)

valid_candidate3["balance_matrix_rank_ranking"] = valid_candidate3.groupby(
    "session"
).balance_matrix_rank.rank()
valid_candidate3["balance_matrix_rank_ranking"] = valid_candidate3[
    "balance_matrix_rank_ranking"
].astype(np.float32)

valid_candidate5["BPR_score_ranking"] = valid_candidate5.groupby(
    "session"
).BPR_score.rank()
valid_candidate5["BPR_score_ranking"] = valid_candidate5["BPR_score_ranking"].astype(
    np.float32
)

valid_candidate6["SRGNN_score_ranking"] = valid_candidate6.groupby(
    "session"
).SRGNN_score.rank()
valid_candidate6["SRGNN_score_ranking"] = valid_candidate6[
    "SRGNN_score_ranking"
].astype(np.float32)

valid_candidate8["item_cf_count_ranking"] = valid_candidate8.groupby(
    "session"
).item_cf_count.rank()
valid_candidate8["item_cf_count_ranking"] = valid_candidate8[
    "item_cf_count_ranking"
].astype(np.float32)

valid_candidate9["time_decay_matrix_count_ranking"] = valid_candidate9.groupby(
    "session"
).time_decay_matrix_count.rank()
valid_candidate9["time_decay_matrix_count_ranking"] = valid_candidate9[
    "time_decay_matrix_count_ranking"
].astype(np.float32)

valid_candidate10["imf_scores_ranking"] = valid_candidate10.groupby(
    "session"
).imf_scores.rank()
valid_candidate10["imf_scores_ranking"] = valid_candidate10[
    "imf_scores_ranking"
].astype(np.float32)

valid_candidate11["w2v_angular_distance_ranking"] = valid_candidate11.groupby(
    "session"
).w2v_angular_distance.rank()
valid_candidate11["w2v_angular_distance_ranking"] = valid_candidate11[
    "w2v_angular_distance_ranking"
].astype(np.float32)

valid_candidate12["user_cf_score_ranking"] = valid_candidate12.groupby(
    "session"
).user_cf_score.rank()
valid_candidate12["user_cf_score_ranking"] = valid_candidate12[
    "user_cf_score_ranking"
].astype(np.float32)].astype(np.float32)

valid_candidate8["item_cf_count_ranking"] = valid_candidate8.groupby(
    "session"
).item_cf_count.rank()
valid_candidate8["item_cf_count_ranking"] = valid_candidate8[
    "item_cf_count_ranking"
].astype(np.float32)

valid_candidate9["time_decay_matrix_count_ranking"] = valid_candidate9.groupby(
    "session"
).time_decay_matrix_count.rank()
valid_candidate9["time_decay_matrix_count_ranking"] = valid_candidate9[
    "time_decay_matrix_count_ranking"
].astype(np.float32)

valid_candidate10["imf_scores_ranking"] = valid_candidate10.groupby(
    "session"
).imf_scores.rank()
valid_candidate10["imf_scores_ranking"] = valid_candidate10[
    "imf_scores_ranking"
].astype(np.float32)

valid_candidate11["w2v_angular_distance_ranking"] = valid_candidate11.groupby(
    "session"
).w2v_angular_distance.rank()
valid_candidate11["w2v_angular_distance_ranking"] = valid_candidate11[
    "w2v_angular_distance_ranking"
].astype(np.float32)

valid_candidate12["user_cf_score_ranking"] = valid_candidate12.groupby(
    "session"
).user_cf_score.rank()
valid_candidate12["user_cf_score_ranking"] = valid_candidate12[
    "user_cf_score_ranking"
].astype(np.float32)].astype(np.float32)

valid_candidate8["item_cf_count_ranking"] = valid_candidate8.groupby(
    "session"
).item_cf_count.rank()
valid_candidate8["item_cf_count_ranking"] = valid_candidate8[
    "item_cf_count_ranking"
].astype(np.float32)

valid_candidate9["time_decay_matrix_count_ranking"] = valid_candidate9.groupby(
    "session"
).time_decay_matrix_count.rank()
valid_candidate9["time_decay_matrix_count_ranking"] = valid_candidate9[
    "time_decay_matrix_count_ranking"
].astype(np.float32)

valid_candidate10["imf_scores_ranking"] = valid_candidate10.groupby(
    "session"
).imf_scores.rank()
valid_candidate10["imf_scores_ranking"] = valid_candidate10[
    "imf_scores_ranking"
].astype(np.float32)

valid_candidate11["w2v_angular_distance_ranking"] = valid_candidate11.groupby(
    "session"
).w2v_angular_distance.rank()
valid_candidate11["w2v_angular_distance_ranking"] = valid_candidate11[
    "w2v_angular_distance_ranking"
].astype(np.float32)

valid_candidate12["user_cf_score_ranking"] = valid_candidate12.groupby(
    "session"
).user_cf_score.rank()
valid_candidate12["user_cf_score_ranking"] = valid_candidate12[
    "user_cf_score_ranking"
].astype(np.float32)].astype(np.float32)

valid_candidate8["item_cf_count_ranking"] = valid_candidate8.groupby(
    "session"
).item_cf_count.rank()
valid_candidate8["item_cf_count_ranking"] = valid_candidate8[
    "item_cf_count_ranking"
].astype(np.float32)

valid_candidate9["time_decay_matrix_count_ranking"] = valid_candidate9.groupby(
    "session"
).time_decay_matrix_count.rank()
valid_candidate9["time_decay_matrix_count_ranking"] = valid_candidate9[
    "time_decay_matrix_count_ranking"
].astype(np.float32)

valid_candidate10["imf_scores_ranking"] = valid_candidate10.groupby(
    "session"
).imf_scores.rank()
valid_candidate10["imf_scores_ranking"] = valid_candidate10[
    "imf_scores_ranking"
].astype(np.float32)

valid_candidate11["w2v_angular_distance_ranking"] = valid_candidate11.groupby(
    "session"
).w2v_angular_distance.rank()
valid_candidate11["w2v_angular_distance_ranking"] = valid_candidate11[
    "w2v_angular_distance_ranking"
].astype(np.float32)

valid_candidate12["user_cf_score_ranking"] = valid_candidate12.groupby(
    "session"
).user_cf_score.rank()
valid_candidate12["user_cf_score_ranking"] = valid_candidate12[
    "user_cf_score_ranking"
].astype(np.float32)

In [ ]:
valid_candidate1 = pl.from_pandas(valid_candidate1)
valid_candidate2 = pl.from_pandas(valid_candidate2)
valid_candidate3 = pl.from_pandas(valid_candidate3)
valid_candidate4 = pl.from_pandas(valid_candidate4)
valid_candidate5 = pl.from_pandas(valid_candidate5)
valid_candidate6 = pl.from_pandas(valid_candidate6)
valid_candidate7 = pl.from_pandas(valid_candidate7)
valid_candidate8 = pl.from_pandas(valid_candidate8)
valid_candidate9 = pl.from_pandas(valid_candidate9)
valid_candidate10 = pl.from_pandas(valid_candidate10)
valid_candidate11 = pl.from_pandas(valid_candidate11)
valid_candidate12 = pl.from_pandas(valid_candidate12)

In [ ]:
def cast_cols(df, columns):
    for col in columns:
        df = df.with_column(df.get_column(col).cast(pl.Int32))
    return df


valid_candidate1 = cast_cols(valid_candidate1, ["session", "aid"])
valid_candidate2 = cast_cols(valid_candidate2, ["session", "aid"])
valid_candidate3 = cast_cols(valid_candidate3, ["session", "aid"])
valid_candidate4 = cast_cols(valid_candidate4, ["session", "aid"])
valid_candidate5 = cast_cols(valid_candidate5, ["session", "aid"])
valid_candidate6 = cast_cols(valid_candidate6, ["session", "aid"])
valid_candidate7 = cast_cols(valid_candidate7, ["session", "aid"])
valid_candidate8 = cast_cols(valid_candidate8, ["session", "aid"])
valid_candidate9 = cast_cols(valid_candidate9, ["session", "aid"])
valid_candidate10 = cast_cols(valid_candidate10, ["session", "aid"])
valid_candidate11 = cast_cols(valid_candidate11, ["session", "aid"])
valid_candidate12 = cast_cols(valid_candidate12, ["session", "aid"])

In [ ]:
valid_candidates = valid_candidate1.join(
    valid_candidate2, on=["session", "aid"], how="outer"
)
del valid_candidate1, valid_candidate2
gc.collect()
print("end")

valid_candidates = valid_candidates.join(
    valid_candidate3, on=["session", "aid"], how="outer"
)
del valid_candidate3
gc.collect()
print("end")

valid_candidates = valid_candidates.join(
    valid_candidate4, on=["session", "aid"], how="outer"
)
del valid_candidate4
gc.collect()
print("end")

valid_candidates = valid_candidates.join(
    valid_candidate5, on=["session", "aid"], how="outer"
)
del valid_candidate5
gc.collect()
print("end")

valid_candidates = valid_candidates.join(
    valid_candidate6, on=["session", "aid"], how="outer"
)
del valid_candidate6
gc.collect()
print("end")

valid_candidates = valid_candidates.join(
    valid_candidate7, on=["session", "aid"], how="outer"
)
del valid_candidate7
gc.collect()
print("end")

valid_candidates = valid_candidates.join(
    valid_candidate8, on=["session", "aid"], how="outer"
)
del valid_candidate8
gc.collect()
print("end")

valid_candidates = valid_candidates.join(
    valid_candidate9, on=["session", "aid"], how="outer"
)
del valid_candidate9
gc.collect()
print("end")

valid_candidates = valid_candidates.join(
    valid_candidate10, on=["session", "aid"], how="outer"
)
del valid_candidate10
gc.collect()
print("end")

valid_candidates = valid_candidates.join(
    valid_candidate11, on=["session", "aid"], how="outer"
)
del valid_candidate11
gc.collect()
print("end")

valid_candidates = valid_candidates.join(
    valid_candidate12, on=["session", "aid"], how="outer"
)
del valid_candidate12
gc.collect()
print("end")print("end")

valid_candidates = valid_candidates.join(
    valid_candidate10, on=["session", "aid"], how="outer"
)
del valid_candidate10
gc.collect()
print("end")

valid_candidates = valid_candidates.join(
    valid_candidate11, on=["session", "aid"], how="outer"
)
del valid_candidate11
gc.collect()
print("end")

valid_candidates = valid_candidates.join(
    valid_candidate12, on=["session", "aid"], how="outer"
)
del valid_candidate12
gc.collect()
print("end")print("end")

valid_candidates = valid_candidates.join(
    valid_candidate10, on=["session", "aid"], how="outer"
)
del valid_candidate10
gc.collect()
print("end")

valid_candidates = valid_candidates.join(
    valid_candidate11, on=["session", "aid"], how="outer"
)
del valid_candidate11
gc.collect()
print("end")

valid_candidates = valid_candidates.join(
    valid_candidate12, on=["session", "aid"], how="outer"
)
del valid_candidate12
gc.collect()
print("end")print("end")

valid_candidates = valid_candidates.join(
    valid_candidate10, on=["session", "aid"], how="outer"
)
del valid_candidate10
gc.collect()
print("end")

valid_candidates = valid_candidates.join(
    valid_candidate11, on=["session", "aid"], how="outer"
)
del valid_candidate11
gc.collect()
print("end")

valid_candidates = valid_candidates.join(
    valid_candidate12, on=["session", "aid"], how="outer"
)
del valid_candidate12
gc.collect()
print("end")

In [ ]:
valid_candidates = valid_candidates.to_pandas()

In [ ]:
# 型を変える
valid_candidates.fillna(-999, inplace=True)  # 欠損うめ
valid_candidates = reduce_mem_usage_for_numeric(valid_candidates)

In [ ]:
valid_candidates.to_pickle(f"{OUTPUT_DIR}/valid_cart_order_candidates.pkl")

In [ ]:
# 最高時のcv見てみる。
for type in ["carts", "orders"]:
    print(f"{type}_candidates_recall")
    calc_candidates_recall(valid_candidates, valid_label, type)

### similarityスコアを付与

In [ ]:
valid_candidates = pd.read_pickle(f"{OUTPUT_DIR}/valid_cart_order_candidates.pkl")

In [ ]:
last_items = (
    valid_session.groupby("session")[["session", "aid"]].tail(1).reset_index(drop=True)
)
last_items = dict(zip(last_items["session"], last_items["aid"]))

last2_items = (
    valid_session.groupby("session")[["session", "aid"]].nth(-2).reset_index(drop=True)
)
last2_items = dict(zip(last2_items["session"], last2_items["aid"]))last2_items = (
    valid_session.groupby("session")[["session", "aid"]].nth(-2).reset_index(drop=True)
)
last2_items = dict(zip(last2_items["session"], last2_items["aid"]))last2_items = (
    valid_session.groupby("session")[["session", "aid"]].nth(-2).reset_index(drop=True)
)
last2_items = dict(zip(last2_items["session"], last2_items["aid"]))last2_items = (
    valid_session.groupby("session")[["session", "aid"]].nth(-2).reset_index(drop=True)
)
last2_items = dict(zip(last2_items["session"], last2_items["aid"]))

In [ ]:
# 共起
matrix_file = f"{ROOT}/data/output/candidates/covisitation_matrix/train/exp/026/time_decay_matrix2_weight_click1_cart1_order1_chunk100000.pkl"
with open(matrix_file, "rb") as f:
    next_AIDs = pickle.load(f)

matrix_scores = []
for session, candidate in tqdm(
    zip(valid_candidates["session"], valid_candidates["aid"]),
    total=len(valid_candidates),
):
    score = next_AIDs[last_items[session]][candidate]
    matrix_scores.append(score)
valid_candidates["time_decay_matrix_score"] = matrix_scores

matrix_scores = []
for session, candidate in tqdm(
    zip(valid_candidates["session"], valid_candidates["aid"]),
    total=len(valid_candidates),
):
    try:
        score = next_AIDs[last2_items[session]][candidate]
    except:
        score = -1
    matrix_scores.append(score)
valid_candidates["time_decay_matrix_score_last2"] = matrix_scores

del next_AIDs
gc.collect()        score = -1
    matrix_scores.append(score)
valid_candidates["time_decay_matrix_score_last2"] = matrix_scores

del next_AIDs
gc.collect()        score = -1
    matrix_scores.append(score)
valid_candidates["time_decay_matrix_score_last2"] = matrix_scores

del next_AIDs
gc.collect()        score = -1
    matrix_scores.append(score)
valid_candidates["time_decay_matrix_score_last2"] = matrix_scores

del next_AIDs
gc.collect()

In [ ]:
# itemcf
itemcf_file = f"{ROOT}/data/output/candidates/itemCF/train/exp/001/sim_item_corr.pkl"
with open(itemcf_file, "rb") as f:
    item_cf_sim = pickle.load(f)

cf_scores = []
for session, candidate in tqdm(
    zip(valid_candidates["session"], valid_candidates["aid"]),
    total=len(valid_candidates),
):
    score = item_cf_sim[last_items[session]][candidate]
    cf_scores.append(score)
valid_candidates["item_cf_score"] = cf_scores

# cf feature
cf_scores = []
for session, candidate in tqdm(
    zip(valid_candidates["session"], valid_candidates["aid"]),
    total=len(valid_candidates),
):
    try:
        score = item_cf_sim[last2_items[session]][candidate]
    except:
        score = -1
    cf_scores.append(score)
valid_candidates["item_cf_score_last2"] = cf_scores

del item_cf_sim
gc.collect()        score = -1
    cf_scores.append(score)
valid_candidates["item_cf_score_last2"] = cf_scores

del item_cf_sim
gc.collect()        score = -1
    cf_scores.append(score)
valid_candidates["item_cf_score_last2"] = cf_scores

del item_cf_sim
gc.collect()        score = -1
    cf_scores.append(score)
valid_candidates["item_cf_score_last2"] = cf_scores

del item_cf_sim
gc.collect()

In [ ]:
model = Word2Vec.load(f"{ROOT}/data/output/candidates/w2v/train/exp/008/word2vec.model")


def cos_sim(v1, v2):
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))


similaritys = []
for session, candidate in tqdm(
    zip(valid_candidates["session"], valid_candidates["aid"]),
    total=len(valid_candidates),
):
    similarity = cos_sim(model.wv[last_items[session]], model.wv[candidate])
    similaritys.append(similarity)

valid_candidates["w2v_similarity"] = similaritys    similaritys.append(similarity)

valid_candidates["w2v_similarity"] = similaritys    similaritys.append(similarity)

valid_candidates["w2v_similarity"] = similaritys    similaritys.append(similarity)

valid_candidates["w2v_similarity"] = similaritys

### 保存

In [ ]:
# castする
valid_candidates = reduce_mem_usage_for_numeric(valid_candidates)
valid_candidates.to_pickle(f"{OUTPUT_DIR}/valid_cart_order_candidates.pkl")

# make candidates test

## click

In [ ]:
candidate1 = pd.read_pickle(
    f"{ROOT}/data/output/candidates/GRU4Rec/test/exp/001/gru4rec_candidates.pkl"
)
candidate2 = pd.read_pickle(
    f"{ROOT}/data/output/candidates/past_events/test/exp/005/past_action_candidates_max100.pkl"
)
candidate3 = pd.read_pickle(
    f"{ROOT}/data/output/candidates/covisitation_matrix/test/click_only_matrix_candidates_max40.pkl"
)
candidate4 = pd.read_pickle(
    f"{ROOT}/data/output/candidates/covisitation_matrix/test/exp/026/time_decay_matrix_exp026_candidates_last10_suggest20.pkl"
)

candidate5 = pd.read_pickle(
    f"{ROOT}/data/output/candidates/BPR/test/exp/005/BPR_candidates.pkl"
)

candidate6 = pd.read_pickle(
    f"{ROOT}/data/output/candidates/SRGNN/test/exp/001/SRGNN_candidates.pkl"
)

candidate7 = pd.read_pickle(
    f"{ROOT}/data/output/candidates/covisitation_matrix/test/exp/011/time_decay_matrix3_weight_click0.5_cart9_order0.5_candidates_last10_suggest20.pkl"
)

candidate8 = pd.read_pickle(
    f"{ROOT}/data/output/candidates/itemCF/test/exp/003/item_cf_candidates_max40.pkl"
)

candidate9 = pd.read_pickle(
    f"{ROOT}/data/output/candidates/covisitation_matrix/test/exp/009/time_decay_matrix_candidates_max40.pkl"
)

candidate10 = pd.read_pickle(
    f"{ROOT}/data/output/candidates/IMF/test/exp/001/imf_candidates.pkl"
)

candidate11 = pd.read_pickle(
    f"{ROOT}/data/output/candidates/w2v/test/exp/008/w2v_candidates.pkl"
)

candidate12 = pd.read_pickle(
    f"{ROOT}/data/output/candidates/user_cf/test/exp/002/user_cf_candidates.pkl"
)
candidate8 = pd.read_pickle(
    f"{ROOT}/data/output/candidates/itemCF/test/exp/003/item_cf_candidates_max40.pkl"
)

candidate9 = pd.read_pickle(
    f"{ROOT}/data/output/candidates/covisitation_matrix/test/exp/009/time_decay_matrix_candidates_max40.pkl"
)

candidate10 = pd.read_pickle(
    f"{ROOT}/data/output/candidates/IMF/test/exp/001/imf_candidates.pkl"
)

candidate11 = pd.read_pickle(
    f"{ROOT}/data/output/candidates/w2v/test/exp/008/w2v_candidates.pkl"
)

candidate12 = pd.read_pickle(
    f"{ROOT}/data/output/candidates/user_cf/test/exp/002/user_cf_candidates.pkl"
)
candidate8 = pd.read_pickle(
    f"{ROOT}/data/output/candidates/itemCF/test/exp/003/item_cf_candidates_max40.pkl"
)

candidate9 = pd.read_pickle(
    f"{ROOT}/data/output/candidates/covisitation_matrix/test/exp/009/time_decay_matrix_candidates_max40.pkl"
)

candidate10 = pd.read_pickle(
    f"{ROOT}/data/output/candidates/IMF/test/exp/001/imf_candidates.pkl"
)

candidate11 = pd.read_pickle(
    f"{ROOT}/data/output/candidates/w2v/test/exp/008/w2v_candidates.pkl"
)

candidate12 = pd.read_pickle(
    f"{ROOT}/data/output/candidates/user_cf/test/exp/002/user_cf_candidates.pkl"
)
candidate8 = pd.read_pickle(
    f"{ROOT}/data/output/candidates/itemCF/test/exp/003/item_cf_candidates_max40.pkl"
)

candidate9 = pd.read_pickle(
    f"{ROOT}/data/output/candidates/covisitation_matrix/test/exp/009/time_decay_matrix_candidates_max40.pkl"
)

candidate10 = pd.read_pickle(
    f"{ROOT}/data/output/candidates/IMF/test/exp/001/imf_candidates.pkl"
)

candidate11 = pd.read_pickle(
    f"{ROOT}/data/output/candidates/w2v/test/exp/008/w2v_candidates.pkl"
)

candidate12 = pd.read_pickle(
    f"{ROOT}/data/output/candidates/user_cf/test/exp/002/user_cf_candidates.pkl"
)

In [ ]:
# candidate_rank
candidate1["gru4rec_score_ranking"] = candidate1.groupby("session").gru4rec_score.rank()
candidate1["gru4rec_score_ranking"] = candidate1["gru4rec_score_ranking"].astype(
    np.float32
)

candidate3["click_only_matrix_rank_ranking"] = candidate3.groupby(
    "session"
).click_only_matrix_rank.rank()
candidate3["click_only_matrix_rank_ranking"] = candidate3[
    "click_only_matrix_rank_ranking"
].astype(np.float32)

candidate5["BPR_score_ranking"] = candidate5.groupby("session").BPR_score.rank()
candidate5["BPR_score_ranking"] = candidate5["BPR_score_ranking"].astype(np.float32)

candidate6["SRGNN_score_ranking"] = candidate6.groupby("session").SRGNN_score.rank()
candidate6["SRGNN_score_ranking"] = candidate6["SRGNN_score_ranking"].astype(np.float32)

candidate8["item_cf_count_ranking"] = candidate8.groupby("session").item_cf_count.rank()
candidate8["item_cf_count_ranking"] = candidate8["item_cf_count_ranking"].astype(
    np.float32
)

candidate9["time_decay_matrix_count_ranking"] = candidate9.groupby(
    "session"
).time_decay_matrix_count.rank()
candidate9["time_decay_matrix_count_ranking"] = candidate9[
    "time_decay_matrix_count_ranking"
].astype(np.float32)

candidate10["imf_scores_ranking"] = candidate10.groupby("session").imf_scores.rank()
candidate10["imf_scores_ranking"] = candidate10["imf_scores_ranking"].astype(np.float32)

candidate11["w2v_angular_distance_ranking"] = candidate11.groupby(
    "session"
).w2v_angular_distance.rank()
candidate11["w2v_angular_distance_ranking"] = candidate11[
    "w2v_angular_distance_ranking"
].astype(np.float32)

candidate12["user_cf_score_ranking"] = candidate12.groupby(
    "session"
).user_cf_score.rank()
candidate12["user_cf_score_ranking"] = candidate12["user_cf_score_ranking"].astype(
    np.float32
)candidate9["time_decay_matrix_count_ranking"] = candidate9[
    "time_decay_matrix_count_ranking"
].astype(np.float32)

candidate10["imf_scores_ranking"] = candidate10.groupby("session").imf_scores.rank()
candidate10["imf_scores_ranking"] = candidate10["imf_scores_ranking"].astype(np.float32)

candidate11["w2v_angular_distance_ranking"] = candidate11.groupby(
    "session"
).w2v_angular_distance.rank()
candidate11["w2v_angular_distance_ranking"] = candidate11[
    "w2v_angular_distance_ranking"
].astype(np.float32)

candidate12["user_cf_score_ranking"] = candidate12.groupby(
    "session"
).user_cf_score.rank()
candidate12["user_cf_score_ranking"] = candidate12["user_cf_score_ranking"].astype(
    np.float32
)candidate9["time_decay_matrix_count_ranking"] = candidate9[
    "time_decay_matrix_count_ranking"
].astype(np.float32)

candidate10["imf_scores_ranking"] = candidate10.groupby("session").imf_scores.rank()
candidate10["imf_scores_ranking"] = candidate10["imf_scores_ranking"].astype(np.float32)

candidate11["w2v_angular_distance_ranking"] = candidate11.groupby(
    "session"
).w2v_angular_distance.rank()
candidate11["w2v_angular_distance_ranking"] = candidate11[
    "w2v_angular_distance_ranking"
].astype(np.float32)

candidate12["user_cf_score_ranking"] = candidate12.groupby(
    "session"
).user_cf_score.rank()
candidate12["user_cf_score_ranking"] = candidate12["user_cf_score_ranking"].astype(
    np.float32
)candidate9["time_decay_matrix_count_ranking"] = candidate9[
    "time_decay_matrix_count_ranking"
].astype(np.float32)

candidate10["imf_scores_ranking"] = candidate10.groupby("session").imf_scores.rank()
candidate10["imf_scores_ranking"] = candidate10["imf_scores_ranking"].astype(np.float32)

candidate11["w2v_angular_distance_ranking"] = candidate11.groupby(
    "session"
).w2v_angular_distance.rank()
candidate11["w2v_angular_distance_ranking"] = candidate11[
    "w2v_angular_distance_ranking"
].astype(np.float32)

candidate12["user_cf_score_ranking"] = candidate12.groupby(
    "session"
).user_cf_score.rank()
candidate12["user_cf_score_ranking"] = candidate12["user_cf_score_ranking"].astype(
    np.float32
)

In [ ]:
candidate1 = pl.from_pandas(candidate1)
candidate2 = pl.from_pandas(candidate2)
candidate3 = pl.from_pandas(candidate3)
candidate4 = pl.from_pandas(candidate4)
candidate5 = pl.from_pandas(candidate5)
candidate6 = pl.from_pandas(candidate6)
candidate7 = pl.from_pandas(candidate7)
candidate8 = pl.from_pandas(candidate8)
candidate9 = pl.from_pandas(candidate9)
candidate10 = pl.from_pandas(candidate10)
candidate11 = pl.from_pandas(candidate11)
candidate12 = pl.from_pandas(candidate12)

In [ ]:
def cast_cols(df, columns):
    for col in columns:
        df = df.with_column(df.get_column(col).cast(pl.Int32))
    return df


candidate1 = cast_cols(candidate1, ["session", "aid"])
candidate2 = cast_cols(candidate2, ["session", "aid"])
candidate3 = cast_cols(candidate3, ["session", "aid"])
candidate4 = cast_cols(candidate4, ["session", "aid"])
candidate5 = cast_cols(candidate5, ["session", "aid"])
candidate6 = cast_cols(candidate6, ["session", "aid"])
candidate7 = cast_cols(candidate7, ["session", "aid"])
candidate8 = cast_cols(candidate8, ["session", "aid"])
candidate9 = cast_cols(candidate9, ["session", "aid"])
candidate10 = cast_cols(candidate10, ["session", "aid"])
candidate11 = cast_cols(candidate11, ["session", "aid"])
candidate12 = cast_cols(candidate12, ["session", "aid"])

In [ ]:
all_candidates = candidate1.join(candidate2, on=["session", "aid"], how="outer")
del candidate1, candidate2
gc.collect()
print("end")

all_candidates = all_candidates.join(candidate3, on=["session", "aid"], how="outer")
del candidate3
gc.collect()
print("end")

all_candidates = all_candidates.join(candidate4, on=["session", "aid"], how="outer")
del candidate4
gc.collect()
print("end")

all_candidates = all_candidates.join(candidate5, on=["session", "aid"], how="outer")
del candidate5
gc.collect()
print("end")

all_candidates = all_candidates.join(candidate6, on=["session", "aid"], how="outer")
del candidate6
gc.collect()
print("end")

all_candidates = all_candidates.join(candidate7, on=["session", "aid"], how="outer")
del candidate7
gc.collect()
print("end")

all_candidates = all_candidates.join(candidate8, on=["session", "aid"], how="outer")
del candidate8
gc.collect()
print("end")

all_candidates = all_candidates.join(candidate9, on=["session", "aid"], how="outer")
del candidate9
gc.collect()
print("end")

all_candidates = all_candidates.join(candidate10, on=["session", "aid"], how="outer")
del candidate10
gc.collect()
print("end")

all_candidates = all_candidates.join(candidate11, on=["session", "aid"], how="outer")
del candidate11
gc.collect()
print("end")

all_candidates = all_candidates.join(candidate12, on=["session", "aid"], how="outer")
del candidate12
gc.collect()
print("end")

# 型を変える
all_candidates = all_candidates.to_pandas()
all_candidates = all_candidates.fillna(-999)
all_candidates = reduce_mem_usage_for_numeric(all_candidates)

### similarity

In [ ]:
last_items = test.groupby("session")[["session", "aid"]].tail(1).reset_index(drop=True)
last_items = dict(zip(last_items["session"], last_items["aid"]))

last2_items = test.groupby("session")[["session", "aid"]].nth(-2).reset_index(drop=True)
last2_items = dict(zip(last2_items["session"], last2_items["aid"]))

In [ ]:
# 共起
matrix_file = f"{ROOT}/data/output/candidates/covisitation_matrix/test/exp/026/time_decay_matrix2_weight_click1_cart1_order1_chunk100000.pkl"
with open(matrix_file, "rb") as f:
    next_AIDs = pickle.load(f)

matrix_scores = []
for session, candidate in tqdm(
    zip(all_candidates["session"], all_candidates["aid"]), total=len(all_candidates)
):
    score = next_AIDs[last_items[session]][candidate]
    matrix_scores.append(score)
all_candidates["time_decay_matrix_score"] = matrix_scores

matrix_scores = []
for session, candidate in tqdm(
    zip(all_candidates["session"], all_candidates["aid"]), total=len(all_candidates)
):
    try:
        score = next_AIDs[last2_items[session]][candidate]
    except:
        score = -1
    matrix_scores.append(score)
all_candidates["time_decay_matrix_score_last2"] = matrix_scores

del next_AIDs
gc.collect()all_candidates["time_decay_matrix_score_last2"] = matrix_scores

del next_AIDs
gc.collect()all_candidates["time_decay_matrix_score_last2"] = matrix_scores

del next_AIDs
gc.collect()all_candidates["time_decay_matrix_score_last2"] = matrix_scores

del next_AIDs
gc.collect()

In [ ]:
# itemcf
itemcf_file = f"{ROOT}/data/output/candidates/itemCF/test/exp/001/sim_item_corr.pkl"
with open(itemcf_file, "rb") as f:
    item_cf_sim = pickle.load(f)

cf_scores = []
for session, candidate in tqdm(
    zip(all_candidates["session"], all_candidates["aid"]), total=len(all_candidates)
):
    score = item_cf_sim[last_items[session]][candidate]
    cf_scores.append(score)
all_candidates["item_cf_score"] = cf_scores

# cf feature
cf_scores = []
for session, candidate in tqdm(
    zip(all_candidates["session"], all_candidates["aid"]), total=len(all_candidates)
):
    try:
        score = item_cf_sim[last2_items[session]][candidate]
    except:
        score = -1
    cf_scores.append(score)
all_candidates["item_cf_score_last2"] = cf_scores

del item_cf_sim
gc.collect()all_candidates["item_cf_score_last2"] = cf_scores

del item_cf_sim
gc.collect()all_candidates["item_cf_score_last2"] = cf_scores

del item_cf_sim
gc.collect()all_candidates["item_cf_score_last2"] = cf_scores

del item_cf_sim
gc.collect()

In [ ]:
# similarity

model = Word2Vec.load(f"{ROOT}/data/output/candidates/w2v/test/exp/008/word2vec.model")


def cos_sim(v1, v2):
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))


similaritys = []
for session, candidate in tqdm(
    zip(all_candidates["session"], all_candidates["aid"]), total=len(all_candidates)
):
    similarity = cos_sim(model.wv[last_items[session]], model.wv[candidate])
    similaritys.append(similarity)

all_candidates["w2v_similarity"] = similaritys
all_candidates["w2v_similarity"] = similaritys
all_candidates["w2v_similarity"] = similaritys
all_candidates["w2v_similarity"] = similaritys

### 保存

In [ ]:
# castする
all_candidates = reduce_mem_usage_for_numeric(all_candidates)
all_candidates.to_pickle(f"{OUTPUT_DIR}/test_click_candidates.pkl")

In [ ]:
del all_candidates
gc.collect()

## cart/order

In [ ]:
candidate1 = pd.read_pickle(
    f"{ROOT}/data/output/candidates/GRU4Rec/test/exp/001/gru4rec_candidates.pkl"
)
candidate2 = pd.read_pickle(
    f"{ROOT}/data/output/candidates/past_events/test/exp/005/past_action_candidates_max100.pkl"
)
candidate3 = pd.read_pickle(
    f"{ROOT}/data/output/candidates/covisitation_matrix/test/balance_matrix_candidates_max40.pkl"
)
candidate4 = pd.read_pickle(
    f"{ROOT}/data/output/candidates/covisitation_matrix/test/exp/026/time_decay_matrix_exp026_candidates_last10_suggest20.pkl"
)

candidate5 = pd.read_pickle(
    f"{ROOT}/data/output/candidates/BPR/test/exp/005/BPR_candidates.pkl"
)

candidate6 = pd.read_pickle(
    f"{ROOT}/data/output/candidates/SRGNN/test/exp/001/SRGNN_candidates.pkl"
)

candidate7 = pd.read_pickle(
    f"{ROOT}/data/output/candidates/covisitation_matrix/test/exp/011/time_decay_matrix3_weight_click0.5_cart9_order0.5_candidates_last10_suggest20.pkl"
)

candidate8 = pd.read_pickle(
    f"{ROOT}/data/output/candidates/itemCF/test/exp/003/item_cf_candidates_max40.pkl"
)

candidate9 = pd.read_pickle(
    f"{ROOT}/data/output/candidates/covisitation_matrix/test/exp/009/time_decay_matrix_candidates_max40.pkl"
)

candidate10 = pd.read_pickle(
    f"{ROOT}/data/output/candidates/IMF/test/exp/001/imf_candidates.pkl"
)

candidate11 = pd.read_pickle(
    f"{ROOT}/data/output/candidates/w2v/test/exp/008/w2v_candidates.pkl"
)

candidate12 = pd.read_pickle(
    f"{ROOT}/data/output/candidates/user_cf/test/exp/002/user_cf_candidates.pkl"
)
candidate8 = pd.read_pickle(
    f"{ROOT}/data/output/candidates/itemCF/test/exp/003/item_cf_candidates_max40.pkl"
)

candidate9 = pd.read_pickle(
    f"{ROOT}/data/output/candidates/covisitation_matrix/test/exp/009/time_decay_matrix_candidates_max40.pkl"
)

candidate10 = pd.read_pickle(
    f"{ROOT}/data/output/candidates/IMF/test/exp/001/imf_candidates.pkl"
)

candidate11 = pd.read_pickle(
    f"{ROOT}/data/output/candidates/w2v/test/exp/008/w2v_candidates.pkl"
)

candidate12 = pd.read_pickle(
    f"{ROOT}/data/output/candidates/user_cf/test/exp/002/user_cf_candidates.pkl"
)
candidate8 = pd.read_pickle(
    f"{ROOT}/data/output/candidates/itemCF/test/exp/003/item_cf_candidates_max40.pkl"
)

candidate9 = pd.read_pickle(
    f"{ROOT}/data/output/candidates/covisitation_matrix/test/exp/009/time_decay_matrix_candidates_max40.pkl"
)

candidate10 = pd.read_pickle(
    f"{ROOT}/data/output/candidates/IMF/test/exp/001/imf_candidates.pkl"
)

candidate11 = pd.read_pickle(
    f"{ROOT}/data/output/candidates/w2v/test/exp/008/w2v_candidates.pkl"
)

candidate12 = pd.read_pickle(
    f"{ROOT}/data/output/candidates/user_cf/test/exp/002/user_cf_candidates.pkl"
)
candidate8 = pd.read_pickle(
    f"{ROOT}/data/output/candidates/itemCF/test/exp/003/item_cf_candidates_max40.pkl"
)

candidate9 = pd.read_pickle(
    f"{ROOT}/data/output/candidates/covisitation_matrix/test/exp/009/time_decay_matrix_candidates_max40.pkl"
)

candidate10 = pd.read_pickle(
    f"{ROOT}/data/output/candidates/IMF/test/exp/001/imf_candidates.pkl"
)

candidate11 = pd.read_pickle(
    f"{ROOT}/data/output/candidates/w2v/test/exp/008/w2v_candidates.pkl"
)

candidate12 = pd.read_pickle(
    f"{ROOT}/data/output/candidates/user_cf/test/exp/002/user_cf_candidates.pkl"
)

In [ ]:
# candidate_rank
candidate1["gru4rec_score_ranking"] = candidate1.groupby("session").gru4rec_score.rank()
candidate1["gru4rec_score_ranking"] = candidate1["gru4rec_score_ranking"].astype(
    np.float32
)

candidate3["balance_matrix_rank_ranking"] = candidate3.groupby(
    "session"
).balance_matrix_rank.rank()
candidate3["balance_matrix_rank_ranking"] = candidate3[
    "balance_matrix_rank_ranking"
].astype(np.float32)

candidate5["BPR_score_ranking"] = candidate5.groupby("session").BPR_score.rank()
candidate5["BPR_score_ranking"] = candidate5["BPR_score_ranking"].astype(np.float32)

candidate6["SRGNN_score_ranking"] = candidate6.groupby("session").SRGNN_score.rank()
candidate6["SRGNN_score_ranking"] = candidate6["SRGNN_score_ranking"].astype(np.float32)

candidate8["item_cf_count_ranking"] = candidate8.groupby("session").item_cf_count.rank()
candidate8["item_cf_count_ranking"] = candidate8["item_cf_count_ranking"].astype(
    np.float32
)

candidate9["time_decay_matrix_count_ranking"] = candidate9.groupby(
    "session"
).time_decay_matrix_count.rank()
candidate9["time_decay_matrix_count_ranking"] = candidate9[
    "time_decay_matrix_count_ranking"
].astype(np.float32)

candidate10["imf_scores_ranking"] = candidate10.groupby("session").imf_scores.rank()
candidate10["imf_scores_ranking"] = candidate10["imf_scores_ranking"].astype(np.float32)

candidate11["w2v_angular_distance_ranking"] = candidate11.groupby(
    "session"
).w2v_angular_distance.rank()
candidate11["w2v_angular_distance_ranking"] = candidate11[
    "w2v_angular_distance_ranking"
].astype(np.float32)

candidate12["user_cf_score_ranking"] = candidate12.groupby(
    "session"
).user_cf_score.rank()
candidate12["user_cf_score_ranking"] = candidate12["user_cf_score_ranking"].astype(
    np.float32
)candidate9["time_decay_matrix_count_ranking"] = candidate9[
    "time_decay_matrix_count_ranking"
].astype(np.float32)

candidate10["imf_scores_ranking"] = candidate10.groupby("session").imf_scores.rank()
candidate10["imf_scores_ranking"] = candidate10["imf_scores_ranking"].astype(np.float32)

candidate11["w2v_angular_distance_ranking"] = candidate11.groupby(
    "session"
).w2v_angular_distance.rank()
candidate11["w2v_angular_distance_ranking"] = candidate11[
    "w2v_angular_distance_ranking"
].astype(np.float32)

candidate12["user_cf_score_ranking"] = candidate12.groupby(
    "session"
).user_cf_score.rank()
candidate12["user_cf_score_ranking"] = candidate12["user_cf_score_ranking"].astype(
    np.float32
)candidate9["time_decay_matrix_count_ranking"] = candidate9[
    "time_decay_matrix_count_ranking"
].astype(np.float32)

candidate10["imf_scores_ranking"] = candidate10.groupby("session").imf_scores.rank()
candidate10["imf_scores_ranking"] = candidate10["imf_scores_ranking"].astype(np.float32)

candidate11["w2v_angular_distance_ranking"] = candidate11.groupby(
    "session"
).w2v_angular_distance.rank()
candidate11["w2v_angular_distance_ranking"] = candidate11[
    "w2v_angular_distance_ranking"
].astype(np.float32)

candidate12["user_cf_score_ranking"] = candidate12.groupby(
    "session"
).user_cf_score.rank()
candidate12["user_cf_score_ranking"] = candidate12["user_cf_score_ranking"].astype(
    np.float32
)candidate9["time_decay_matrix_count_ranking"] = candidate9[
    "time_decay_matrix_count_ranking"
].astype(np.float32)

candidate10["imf_scores_ranking"] = candidate10.groupby("session").imf_scores.rank()
candidate10["imf_scores_ranking"] = candidate10["imf_scores_ranking"].astype(np.float32)

candidate11["w2v_angular_distance_ranking"] = candidate11.groupby(
    "session"
).w2v_angular_distance.rank()
candidate11["w2v_angular_distance_ranking"] = candidate11[
    "w2v_angular_distance_ranking"
].astype(np.float32)

candidate12["user_cf_score_ranking"] = candidate12.groupby(
    "session"
).user_cf_score.rank()
candidate12["user_cf_score_ranking"] = candidate12["user_cf_score_ranking"].astype(
    np.float32
)

In [ ]:
candidate1 = pl.from_pandas(candidate1)
candidate2 = pl.from_pandas(candidate2)
candidate3 = pl.from_pandas(candidate3)
candidate4 = pl.from_pandas(candidate4)
candidate5 = pl.from_pandas(candidate5)
candidate6 = pl.from_pandas(candidate6)
candidate7 = pl.from_pandas(candidate7)
candidate8 = pl.from_pandas(candidate8)
candidate9 = pl.from_pandas(candidate9)
candidate10 = pl.from_pandas(candidate10)
candidate11 = pl.from_pandas(candidate11)
candidate12 = pl.from_pandas(candidate12)

In [ ]:
def cast_cols(df, columns):
    for col in columns:
        df = df.with_column(df.get_column(col).cast(pl.Int32))
    return df


candidate1 = cast_cols(candidate1, ["session", "aid"])
candidate2 = cast_cols(candidate2, ["session", "aid"])
candidate3 = cast_cols(candidate3, ["session", "aid"])
candidate4 = cast_cols(candidate4, ["session", "aid"])
candidate5 = cast_cols(candidate5, ["session", "aid"])
candidate6 = cast_cols(candidate6, ["session", "aid"])
candidate7 = cast_cols(candidate7, ["session", "aid"])
candidate8 = cast_cols(candidate8, ["session", "aid"])
candidate9 = cast_cols(candidate9, ["session", "aid"])
candidate10 = cast_cols(candidate10, ["session", "aid"])
candidate11 = cast_cols(candidate11, ["session", "aid"])
candidate12 = cast_cols(candidate12, ["session", "aid"])

In [ ]:
all_candidates = candidate1.join(candidate2, on=["session", "aid"], how="outer")
del candidate1, candidate2
gc.collect()
print("end")

all_candidates = all_candidates.join(candidate3, on=["session", "aid"], how="outer")
del candidate3
gc.collect()
print("end")

all_candidates = all_candidates.join(candidate4, on=["session", "aid"], how="outer")
del candidate4
gc.collect()
print("end")

all_candidates = all_candidates.join(candidate5, on=["session", "aid"], how="outer")
del candidate5
gc.collect()
print("end")

all_candidates = all_candidates.join(candidate6, on=["session", "aid"], how="outer")
del candidate6
gc.collect()
print("end")

all_candidates = all_candidates.join(candidate7, on=["session", "aid"], how="outer")
del candidate7
gc.collect()
print("end")

all_candidates = all_candidates.join(candidate8, on=["session", "aid"], how="outer")
del candidate8
gc.collect()
print("end")

all_candidates = all_candidates.join(candidate9, on=["session", "aid"], how="outer")
del candidate9
gc.collect()
print("end")

all_candidates = all_candidates.join(candidate10, on=["session", "aid"], how="outer")
del candidate10
gc.collect()
print("end")

all_candidates = all_candidates.join(candidate11, on=["session", "aid"], how="outer")
del candidate11
gc.collect()
print("end")

all_candidates = all_candidates.join(candidate12, on=["session", "aid"], how="outer")
del candidate12
gc.collect()
print("end")

# 型を変える
all_candidates = all_candidates.to_pandas()
all_candidates = all_candidates.fillna(-999)
all_candidates = reduce_mem_usage_for_numeric(all_candidates)

### similarity

In [ ]:
last_items = test.groupby("session")[["session", "aid"]].tail(1).reset_index(drop=True)
last_items = dict(zip(last_items["session"], last_items["aid"]))

last2_items = test.groupby("session")[["session", "aid"]].nth(-2).reset_index(drop=True)
last2_items = dict(zip(last2_items["session"], last2_items["aid"]))

In [ ]:
# 共起
matrix_file = f"{ROOT}/data/output/candidates/covisitation_matrix/test/exp/026/time_decay_matrix2_weight_click1_cart1_order1_chunk100000.pkl"
with open(matrix_file, "rb") as f:
    next_AIDs = pickle.load(f)


matrix_scores = []
for session, candidate in tqdm(
    zip(all_candidates["session"], all_candidates["aid"]), total=len(all_candidates)
):
    score = next_AIDs[last_items[session]][candidate]
    matrix_scores.append(score)
all_candidates["time_decay_matrix_score"] = matrix_scores

matrix_scores = []
for session, candidate in tqdm(
    zip(all_candidates["session"], all_candidates["aid"]), total=len(all_candidates)
):
    try:
        score = next_AIDs[last2_items[session]][candidate]
    except:
        score = -1
    matrix_scores.append(score)
all_candidates["time_decay_matrix_score_last2"] = matrix_scores

del next_AIDs, matrix_scores
gc.collect()all_candidates["time_decay_matrix_score_last2"] = matrix_scores

del next_AIDs, matrix_scores
gc.collect()all_candidates["time_decay_matrix_score_last2"] = matrix_scores

del next_AIDs, matrix_scores
gc.collect()all_candidates["time_decay_matrix_score_last2"] = matrix_scores

del next_AIDs, matrix_scores
gc.collect()

In [ ]:
# itemcf
itemcf_file = f"{ROOT}/data/output/candidates/itemCF/test/exp/001/sim_item_corr.pkl"
with open(itemcf_file, "rb") as f:
    item_cf_sim = pickle.load(f)

cf_scores = []
for session, candidate in tqdm(
    zip(all_candidates["session"], all_candidates["aid"]), total=len(all_candidates)
):
    score = item_cf_sim[last_items[session]][candidate]
    cf_scores.append(score)
all_candidates["item_cf_score"] = cf_scores

# cf feature
cf_scores = []
for session, candidate in tqdm(
    zip(all_candidates["session"], all_candidates["aid"]), total=len(all_candidates)
):
    try:
        score = item_cf_sim[last2_items[session]][candidate]
    except:
        score = -1
    cf_scores.append(score)
all_candidates["item_cf_score_last2"] = cf_scores

del item_cf_sim, cf_scores
gc.collect()all_candidates["item_cf_score_last2"] = cf_scores

del item_cf_sim, cf_scores
gc.collect()all_candidates["item_cf_score_last2"] = cf_scores

del item_cf_sim, cf_scores
gc.collect()all_candidates["item_cf_score_last2"] = cf_scores

del item_cf_sim, cf_scores
gc.collect()

In [ ]:
# similarity

model = Word2Vec.load(f"{ROOT}/data/output/candidates/w2v/test/exp/008/word2vec.model")


def cos_sim(v1, v2):
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))


similaritys = []
for session, candidate in tqdm(
    zip(all_candidates["session"], all_candidates["aid"]), total=len(all_candidates)
):
    similarity = cos_sim(model.wv[last_items[session]], model.wv[candidate])
    similaritys.append(similarity)

all_candidates["w2v_similarity"] = similaritys

del similaritys
gc.collect()del similaritys
gc.collect()del similaritys
gc.collect()del similaritys
gc.collect()

### 保存

In [ ]:
# castする
all_candidates = reduce_mem_usage_for_numeric(all_candidates)
all_candidates.to_pickle(f"{OUTPUT_DIR}/test_cart_order_candidates.pkl")